In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, sys
PATH = os.path.abspath("")[:-3]

In [2]:

# SCRAPPEAR UN USUARIO
from test_user import get_user_data

USER = "ComicGirlAshley"
user_data = get_user_data(USER)
print(user_data)

[340, False, 26626, 97, 19968, 134, 6255, 4.637254901960785, 17.470588235294116, 271.45098039215685, 0.8921568627450981, 0.1568627450980392, 26529]
[340, False, 26626, 97, 19968, 134, 6255, 4.637254901960785, 17.470588235294116, 271.45098039215685, 0.8921568627450981, 0.1568627450980392, 26529]


c:\Users\maria\Documents\Uni\org_ia\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
# Se leen los datos generados con model.py\n",
df_data = pd.read_csv(PATH + "data/training_new.csv").sample(frac=1).reset_index(drop=True)
data = df_data.iloc[:, 1:]
print(data.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0          54         0              24            78          327   
1         445         0              78             3         8303   
2           1         0             746           999       158338   
3          55         0             392           383        62776   
4           6         0              82           217         5319   

   listedCount  mediaCount  avg_reply  avg_retweet  like_reply  avg_quote  \
0            0           0   0.000000     0.235294    0.323529        0.0   
1           11           0   0.000000     0.020408    0.061224        0.0   
2            1          52   0.000000     0.000000    0.000000        0.0   
3            6         227   0.078947     0.000000    0.447368        0.0   
4            2         439   0.000000     0.000000    0.000000        0.0   

   hora_rep  followers_diff  clase  
0  0.176471             -24    bot  
1  0.285714             -78    bot  
2  1.

In [4]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
header_names = list(data.columns.values)
for col in data.iloc[:,:-1]:
    data[col] = preprocessing.MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))
x_data = data.iloc[:,:-1].to_numpy()
y_data = data.iloc[:,-1].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.33)

In [5]:
#CONVERTIRLA SALIDA A UN VECTOR DE 0 Y 1
from sklearn.preprocessing import LabelBinarizer

encoder=LabelBinarizer()
label = encoder.fit_transform(y_train)
y_train_transformed = np.hstack((label, 1 - label))
label = encoder.fit_transform(y_test)
y_test_transformed = np.hstack((label, 1 - label))
print(y_train[:5])
print(y_train_transformed[:5])

['human' 'bot' 'bot' 'human' 'human']
[[1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]]


In [6]:
# COMPROBAR TIPO DE LOS DATOS
print(type(x_train))
print(type(y_train_transformed))
# COMPROBAR DIMENSIONES DE LOS DATOS
print(x_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test_transformed.shape) 
# OBTENER DIMENSION DE LA ENTRADA Y NÚMERO DE SALIDAS
input_shape = (x_train.shape[1] ,)
num_clases = y_test_transformed.shape[1]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1616, 13)
(1616,)
(1616, 2)
(796, 13)
(796,)
(796, 2)


In [16]:
# Crear modelo usando sigmoide
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten
input_shape= (x_train.shape[1] ,)
model = Sequential()
arch = [20, 20, 0]
model.add(Dense(arch[0], input_shape=input_shape, activation='sigmoid'))
model.add(Dense(arch[1], input_shape=input_shape, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 20)                280       
                                                                 
 dense_3 (Dense)             (None, 20)                420       
                                                                 
 dense_4 (Dense)             (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________


In [17]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping
lr = 0.3
#batch_size=32
epochs = 80000
early_stopping = EarlyStopping(monitor='val_loss', patience=1)
#Para poder ver la curva de validación hay que poner validation_freq=1. Tarda más
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, validation_split=0.2,
    shuffle=False, validation_freq=1, batch_size=16, callbacks=[early_stopping])

Epoch 1/80000
81/81 [==============================] - 1s 7ms/step - loss: 0.2524 - accuracy: 0.5449 - mse: 0.2524 - val_loss: 0.2342 - val_accuracy: 0.6235 - val_mse: 0.2342
Epoch 2/80000
81/81 [==============================] - 0s 3ms/step - loss: 0.2479 - accuracy: 0.5565 - mse: 0.2479 - val_loss: 0.2325 - val_accuracy: 0.6235 - val_mse: 0.2325
Epoch 3/80000
81/81 [==============================] - 0s 3ms/step - loss: 0.2462 - accuracy: 0.5596 - mse: 0.2462 - val_loss: 0.2305 - val_accuracy: 0.6235 - val_mse: 0.2305
Epoch 4/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.2442 - accuracy: 0.5650 - mse: 0.2442 - val_loss: 0.2280 - val_accuracy: 0.6235 - val_mse: 0.2280
Epoch 5/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.2417 - accuracy: 0.5751 - mse: 0.2417 - val_loss: 0.2247 - val_accuracy: 0.6235 - val_mse: 0.2247
Epoch 6/80000
81/81 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.5967 - mse: 0.2385 - val_loss

In [19]:
col_names = ['arquitectura', 'loss', 'val_loss', 'accuracy', 'val_accuracy', 'epoch']
#dff = pd.DataFrame([], columns=col_names)
dff = pd.read_csv(PATH + 'data/vals.csv')
final_epoch=len(historico.history['loss'])-1
print(final_epoch)
tloss = historico.history['loss'][final_epoch-1]
print(tloss)
vloss = historico.history['val_loss'][final_epoch-1]
print(vloss)
tacc = historico.history['accuracy'][final_epoch-1]
print(tacc)
vacc = historico.history['val_accuracy'][final_epoch-1]
print(vacc)
datos = pd.DataFrame([[arch, tloss, vloss, tacc, vacc, final_epoch]], columns=col_names)
datos = pd.concat([dff, datos], ignore_index=True)
datos.to_csv(PATH + 'data/vals.csv', index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/vals.csv'